# BASE CODE

In [6]:
# Importing the libraries
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Extra libraries
import json
import googlemaps
from datetime import datetime, timedelta

with open('creds.json') as f:
    api_key = json.load(f)

use_api = False
if (use_api == True):
    gmaps = googlemaps.Client(key=api_key['GOOGLE_MAPS_API_KEYS'])
else:
    gmaps = None

# API KEY    
## api_key['GOOGLE_MAPS_API_KEYS']

FileNotFoundError: [Errno 2] No such file or directory: 'creds.json'

In [130]:
df = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv") # OR USE searchData/limitData
df = df.drop_duplicates()
df = df.sample(frac=0.0005, random_state=1) # GETS SMALL SMAPLE OF ROWS
df = df.reset_index()
print(df.dtypes)
print("Length:", len(df))
df.head(2)

index                    int64
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object
Length: 74


,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,102277,2021-07,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121.0,Improved,1986,63 years 10 months,580000.0
1,44647,2019-02,TAMPINES,4 ROOM,864A,TAMPINES ST 83,13 TO 15,104.0,Model A,1988,68 years 06 months,450000.0


In [88]:
df['year'] = df['month'].apply(lambda x: x.split('-')[0]).astype(int)
df['month'] = df['month'].apply(lambda x: x.split('-')[1]).astype(int)

# Convert remaining lease from years to months
df['month_remaining_lease'] = df['remaining_lease'].apply(lambda x: int(x.split(' ')[0]) * 12 + int(x.split(' ')[2]) if len(x.split(' ')) > 2 else int(x.split(' ')[0]) * 12)

# Split storey range into min and max
df['storey_range_low'] = df['storey_range'].apply(lambda x: int(x.split('TO')[0]))
df['storey_range_high'] = df['storey_range'].apply(lambda x: int(x.split('TO')[1]))

# Convert categorical columns to numerical
df['flat_model'] = df['flat_model'].astype('category').cat.codes

df[['month', 'year', 'town', 'flat_type', 'floor_area_sqm', 'lease_commence_date', 'month_remaining_lease', 'flat_model', 'storey_range_low', 'storey_range_high', 'resale_price']].head(5)

,month,year,town,flat_type,floor_area_sqm,lease_commence_date,month_remaining_lease,flat_model,storey_range_low,storey_range_high,resale_price
0,7,2021,HOUGANG,5 ROOM,121.0,1986,766,3,10,12,580000.0
1,2,2019,TAMPINES,4 ROOM,104.0,1988,822,5,13,15,450000.0
2,7,2021,BUKIT BATOK,3 ROOM,73.0,1985,751,5,7,9,357000.0
3,5,2018,PUNGGOL,4 ROOM,92.0,2014,1141,5,13,15,463000.0
4,11,2021,WOODLANDS,4 ROOM,106.0,1995,877,5,4,6,420000.0


# ADD ON COLUMNS CODE

## 1. Get walking distance to nearest MRT station

In [105]:
def get_nearest_mrt_location(addr, location_type):
    # Request directions via public transit
    # Returns Start, End, Total distance

    coords = gmaps.geocode(f"{addr}, Singapore")
    
    lat = coords[0]['geometry']['location']['lat']
    lng = coords[0]['geometry']['location']['lng']

    stn_found = False
    radius = 100

    while not stn_found:
        nearest_location = gmaps.places_nearby(location=f"{lat}, {lng}", keyword=location_type, rank_by="distance")
        if len(nearest_location['results']) > 0:
            stn_found = True
        else:
            radius += 100
    
    if "MRT" in nearest_location['results'][0]['name']:
        if "Singapore" in nearest_location['results'][0]['name']:
            place_addr = place_addr = nearest_location['results'][0]['name']
        else:
            place_addr = place_addr = nearest_location['results'][0]['name'] + ", Singapore"
    else:
        place_addr = place_addr = nearest_location['results'][0]['name'] + " MRT Station, Singapore"
        
    if len(place_addr) == 0:
        raise Exception("No MRT station found")
    
    #print(nearest_location)
    print("START:", f"{addr}")
    print("END:", place_addr)
    
    directions_result = gmaps.distance_matrix(addr, place_addr, mode="walking")
    print("DISTANCE:", directions_result['rows'][0]['elements'][0]['distance']['text'])
    print("\n")

    return  directions_result['rows'][0]['elements'][0]['distance']['text'].split(' ')[0]

In [ ]:
get_nearest_mrt_location("627 PASIR RIS DR 3, Singapore", "mrt")

In [ ]:
df['dist_to_nearest_stn_walking'] = df.apply(lambda row : get_nearest_mrt_location(f"{row['block']} {row['street_name']}, Singapore", "mrt"), axis = 1)

## 2. Get walking distance to nearest supermarket

In [99]:
def get_nearest_sp_location(addr, location_type):
    # Request directions via public transit
    # Returns Start, End, Total distance

    coords = gmaps.geocode(f"{addr}, Singapore")
    
    lat = coords[0]['geometry']['location']['lat']
    lng = coords[0]['geometry']['location']['lng']

    stn_found = False
    radius = 100

    while not stn_found:
        nearest_location = gmaps.places(location=f"{lat}, {lng}", query=location_type, radius=radius)
        if len(nearest_location['results']) > 0:
            stn_found = True
        else:
            radius += 100
    
    directions_result = gmaps.directions(f"{addr}",
                                     nearest_location['results'][0]['formatted_address'],
                                     mode="walking",
                                     departure_time=datetime.now())[0]
    
    '''
    print("START:", f"{addr}")
    print("END:", nearest_location['results'][0]['name'], ",", nearest_location['results'][0]['formatted_address'])
    print("DISTANCE:", directions_result['legs'][0]['distance']['text'])
    print("\n")
    '''
    
    return directions_result['legs'][0]['distance']['text'].split(' ')[0]

In [100]:
df['dist_to_nearest_supermarket_walking'] = df.apply(lambda row : get_nearest_sp_location(f"{row['block']} {row['street_name']}, Singapore", "supermarket"), axis = 1)

In [101]:
df.head(20)

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,month_remaining_lease,storey_range_low,storey_range_high,travel_time_to_town_via_driving,total_population,dist_to_nearest_supermarket_walking
0,102277.0,7.0,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121.0,3.0,1986.0,63 years 10 months,580000.0,2021,766.0,10.0,12.0,21.0,0.946597,0.3
1,101607.0,7.0,BUKIT BATOK,3 ROOM,127,BT BATOK WEST AVE 6,07 TO 09,73.0,5.0,1985.0,62 years 07 months,357000.0,2021,751.0,7.0,9.0,27.0,0.946597,0.6
2,113957.0,11.0,WOODLANDS,4 ROOM,809,WOODLANDS ST 81,04 TO 06,106.0,5.0,1995.0,73 years 01 month,420000.0,2021,877.0,4.0,6.0,32.0,0.946597,0.3
3,114570.0,12.0,BUKIT MERAH,3 ROOM,8,JLN BT HO SWEE,01 TO 03,56.0,9.0,1971.0,48 years 08 months,292000.0,2021,584.0,1.0,3.0,11.0,0.946597,1.0
4,103995.0,8.0,ANG MO KIO,4 ROOM,462,ANG MO KIO AVE 10,01 TO 03,92.0,6.0,1980.0,57 years 06 months,408000.0,2021,690.0,1.0,3.0,18.0,0.946597,0.8
5,113025.0,11.0,PUNGGOL,2 ROOM,209C,PUNGGOL PL,04 TO 06,46.0,0.0,2014.0,92 years 02 months,282000.0,2021,1106.0,4.0,6.0,21.0,0.946597,1.9
6,102966.0,7.0,QUEENSTOWN,4 ROOM,5,HOLLAND CL,07 TO 09,105.0,5.0,1998.0,76 years 05 months,785000.0,2021,917.0,7.0,9.0,18.0,0.946597,12.0
7,99146.0,6.0,BEDOK,4 ROOM,185,BEDOK NTH RD,13 TO 15,92.0,5.0,2005.0,83 years 07 months,608000.0,2021,1003.0,13.0,15.0,19.0,0.946597,0.8
8,97680.0,5.0,HOUGANG,4 ROOM,15,HOUGANG AVE 3,04 TO 06,82.0,3.0,1975.0,53 years 07 months,350000.0,2021,643.0,4.0,6.0,20.0,0.946597,0.6
9,100266.0,6.0,PUNGGOL,4 ROOM,122B,EDGEDALE PLAINS,07 TO 09,93.0,5.0,2017.0,94 years 11 months,470000.0,2021,1139.0,7.0,9.0,22.0,0.946597,0.8


## 3. Get Driving Time Taken to Town (City Hall Mrt)

In [93]:

def get_travel_time_to_town_in_mins(orig_addr, dt=None):
    orig_coords = gmaps.geocode(f"{orig_addr}, Singapore")
    
    orig_lat = orig_coords[0]['geometry']['location']['lat']
    orig_lng = orig_coords[0]['geometry']['location']['lng']
    
    # Town Destination: City hall mrt
    #dest_addr = "150 North Bridge Rd"
    #dest_coords = gmaps.geocode(f"{dest_addr}, Singapore")
    # dest_lat = dest_coords[0]['geometry']['location']['lat'] 
    dest_lat = 1.2931553
    #dest_lng = dest_coords[0]['geometry']['location']['lng']
    dest_lng = 103.852031
    
    if(dt is None):
        dt = datetime.now() # change to monday 8am
        dt = dt.replace(hour=8, minute=0, second=0, microsecond=0 )
        days = (0 - dt.weekday() + 7) % 7
        dt = dt + timedelta(days=days)
    
    driving_directions_result = gmaps.directions(
                                         str(orig_lat)+", "+str(orig_lng),
                                         str(dest_lat)+", "+str(dest_lng),
                                         mode="driving",
                                         avoid="ferries",
                                         departure_time=dt
                                        )
    # transit_directions_result = gmaps.directions(
    #                                      str(orig_lat)+", "+str(orig_lng),
    #                                      str(dest_lat)+", "+str(dest_lng),
    #                                      mode="transit",
    #                                      departure_time=dt
    #                                     )

    
    drive_time_taken = int(int(driving_directions_result[0]['legs'][0]['duration']['value'])/60)
    #print("drive_time_taken:"+str(drive_time_taken))
    
    # if transit_directions_result: #list is not empty
    #     transit_time_taken = int(int(transit_directions_result[0]['legs'][0]['duration']['value'])/60)
    # else: #list is empty
    #     transit_time_taken = int(drive_time_taken * 2.5)
    
    #print("transit_time_taken:"+str(transit_time_taken))

    return  drive_time_taken #, transit_time_taken


In [94]:
get_travel_time_to_town_in_mins("buangkok crescent")

22

In [95]:
dt = datetime.now() # change to monday 8am
dt = dt.replace(hour=8, minute=0, second=0, microsecond=0 )
days = (0 - dt.weekday() + 7) % 7
dt = dt + timedelta(days=days)

df['travel_time_to_town_via_driving'] = df.apply(lambda row : get_travel_time_to_town_in_mins(f"{row['block']} {row['street_name']}, Singapore", dt), axis = 1)

In [96]:
df.head(5)

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,month_remaining_lease,storey_range_low,storey_range_high,travel_time_to_town_via_driving
0,102277,7,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121.0,3,1986,63 years 10 months,580000.0,2021,766,10,12,21
1,44647,2,TAMPINES,4 ROOM,864A,TAMPINES ST 83,13 TO 15,104.0,5,1988,68 years 06 months,450000.0,2019,822,13,15,20
2,101607,7,BUKIT BATOK,3 ROOM,127,BT BATOK WEST AVE 6,07 TO 09,73.0,5,1985,62 years 07 months,357000.0,2021,751,7,9,27
3,27448,5,PUNGGOL,4 ROOM,208B,PUNGGOL PL,13 TO 15,92.0,5,2014,95 years 01 month,463000.0,2018,1141,13,15,21
4,113957,11,WOODLANDS,4 ROOM,809,WOODLANDS ST 81,04 TO 06,106.0,5,1995,73 years 01 month,420000.0,2021,877,4,6,32


## 4. Population size in Singapore for that year

In [127]:
population_df = pd.read_excel('data/sg_population_size_from_singstat.gov.sg.xlsx')  
population_df = population_df[["year","total_population"]]

#normalized using min-max feature scaling
population_df["total_population"] = (population_df["total_population"] - population_df["total_population"].min()) / (population_df["total_population"].max() - population_df["total_population"].min())    

#merge to dataframe based on year
df = pd.merge(df, population_df, how='left')

print(df.dtypes)
df.head(5)

index                                    int64
month                                    int64
town                                    object
flat_type                               object
block                                   object
street_name                             object
storey_range                            object
floor_area_sqm                           int64
flat_model                               int64
lease_commence_date                      int64
remaining_lease                         object
resale_price                             int64
year                                     int64
month_remaining_lease                    int64
storey_range_low                         int64
storey_range_high                        int64
travel_time_to_town_via_driving          int64
total_population                       float64
dist_to_nearest_supermarket_walking    float64
dist_to_nearest_stn_walking            float64
dtype: object


,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,month_remaining_lease,storey_range_low,storey_range_high,travel_time_to_town_via_driving,total_population,dist_to_nearest_supermarket_walking,dist_to_nearest_stn_walking
0,102277,7,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121,3,1986,63 years 10 months,580000,2021,766,10,12,21,0.946597,0.3,0.6
1,101607,7,BUKIT BATOK,3 ROOM,127,BT BATOK WEST AVE 6,07 TO 09,73,5,1985,62 years 07 months,357000,2021,751,7,9,27,0.946597,0.6,0.7
2,113957,11,WOODLANDS,4 ROOM,809,WOODLANDS ST 81,04 TO 06,106,5,1995,73 years 01 month,420000,2021,877,4,6,32,0.946597,0.3,0.9
3,114570,12,BUKIT MERAH,3 ROOM,8,JLN BT HO SWEE,01 TO 03,56,9,1971,48 years 08 months,292000,2021,584,1,3,11,0.946597,1.0,0.5
4,103995,8,ANG MO KIO,4 ROOM,462,ANG MO KIO AVE 10,01 TO 03,92,6,1980,57 years 06 months,408000,2021,690,1,3,18,0.946597,0.8,0.6


In [131]:
#df.to_csv('data_with_added_cols/data_upd_w_dist_travel_time_population3.csv')

## 5. Get PSI reading of the area for month and year

In [30]:
# Importing the libraries
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Extra libraries
import json
import googlemaps
from datetime import datetime, timedelta

api = "eh': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

results = pd.DataFrame()
# Extract only the 'psi_twenty_four_hourly' data from the readings attribute for every month at 0000hrs in the YYYY-MM-DD[T]HH:mm:ss (SGT) format from 2017 to 2023
for year in range(2017, 2024):
    # For every month only get the first day of the month
    for month in range(1, 13):
        # Break out of the loop if the year is 2023 and the month is April
        if month == 4 and year == 2023:
            break
        # Get the first day of the month
        date = datetime(year, month, 1)
        # Get the timestamp in the format YYYY-MM-DD[T]HH:mm:ss (SGT)
        timestamp = date.strftime('%Y-%m-%dT%H:%M:%S')
        # Make the request
        response = requests.get(api, params={'date_time': timestamp})
        # Convert the response to a JSON object
        data = response.json()
        # Extract the readings attribute
        readings = data['items'][0]['readings']
        # Convert the readings attribute to a DataFrame
        df = pd.DataFrame(readings)
        # Extract only the 'psi_twenty_four_hourly' data for each area and rename column to 'psi'
        df = df[['psi_twenty_four_hourly']].rename(columns={'psi_twenty_four_hourly': 'psi'})
        # Create column for area
        df['area'] = df.index
        # Create column and convert month to integer using months_to_integer dictionary
        df['month'] = int(months_to_integer[date.strftime('%B')])
        # Create column for Year as integer
        df['year'] = int(date.strftime('%Y'))
        # Append the data to the results DataFrame
        results = results.append(df)


# Save the data to a file in 'data' directory as csv
results.to_csv('data/psi_data.csv', index=False)


C:\Users\USER\AppData\Local\Temp\ipykernel_4376\3334031408.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df)
C:\Users\USER\AppData\Local\Temp\ipykernel_4376\3334031408.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df)
C:\Users\USER\AppData\Local\Temp\ipykernel_4376\3334031408.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df)
C:\Users\USER\AppData\Local\Temp\ipykernel_4376\3334031408.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df)
C:\Users\USER\AppData\Local\Temp\ipykernel_4376\3334031408.py:46: FutureWarning:

In [32]:
# Read the data from the file and print the first 5 rows
df_psi = pd.read_csv('data/psi_data.csv')

# Print the first 10 rows of the extracted data
print(df_psi.head(10))

   psi      area  month  year
0   39  national      1  2017
1   33     south      1  2017
2   36     north      1  2017
3   39      east      1  2017
4   29   central      1  2017
5   26      west      1  2017
6   39      east      2  2017
7   39   central      2  2017
8   38     south      2  2017
9   36     north      2  2017


## 6. Get Cost of living in Singapore by month and year
#### The Consumer Price Index (CPI) is designed to measure the average price changes in a fixed basket of consumption goods and services commonly purchased by the resident households over time. It is widely used as a measure of the consumer price inflation.
#### The base year is the period with which all the other periods are compared. The base year for the current Consumer Price Index (CPI) is 2019.

In [38]:
url = 'https://data.gov.sg/api/action/datastore_search?resource_id=e7485f4b-eb19-45f2-a853-ec0e97ac1939'

df_cpi = pd.read_csv('data/cpi_data.csv')

# Print the first 10 rows of the extracted data
print(df_cpi.head(1))

    Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0  Data Series  2023 Feb   2023 Jan   2022 Dec   2022 Nov   2022 Oct    

  Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 737 Unnamed: 738  \
0  2022 Sep   2022 Aug   2022 Jul   2022 Jun   ...    1961 Oct     1961 Sep    

  Unnamed: 739 Unnamed: 740 Unnamed: 741 Unnamed: 742 Unnamed: 743  \
0    1961 Aug     1961 Jul     1961 Jun     1961 May     1961 Apr    

  Unnamed: 744 Unnamed: 745      Index  
0    1961 Mar     1961 Feb   1961 Jan   

[1 rows x 747 columns]


## CHECKPOINT: AFTER 1-4 COLUMNS ADDED

In [132]:
df = pd.read_csv("data_with_added_cols/data_upd_w_dist_travel_time_population.csv",index_col=0)
df.head(5)

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,month_remaining_lease,storey_range_low,storey_range_high,travel_time_to_town_via_driving,total_population,dist_to_nearest_supermarket_walking,dist_to_nearest_stn_walking
0,102277,7,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121,3,1986,63 years 10 months,580000,2021,766,10,12,21,0.946597,0.3,0.6
1,101607,7,BUKIT BATOK,3 ROOM,127,BT BATOK WEST AVE 6,07 TO 09,73,5,1985,62 years 07 months,357000,2021,751,7,9,27,0.946597,0.6,0.7
2,113957,11,WOODLANDS,4 ROOM,809,WOODLANDS ST 81,04 TO 06,106,5,1995,73 years 01 month,420000,2021,877,4,6,32,0.946597,0.3,0.9
3,114570,12,BUKIT MERAH,3 ROOM,8,JLN BT HO SWEE,01 TO 03,56,9,1971,48 years 08 months,292000,2021,584,1,3,11,0.946597,1.0,0.5
4,103995,8,ANG MO KIO,4 ROOM,462,ANG MO KIO AVE 10,01 TO 03,92,6,1980,57 years 06 months,408000,2021,690,1,3,18,0.946597,0.8,0.6


## 5. 